# DataMining

**TASK** for each driver, generates the ideal standard route that the driver will have the least divergence.

**Solution** DBSCAN on drivers , take the clustroid of the cluster that maximize the similarity

In [1]:
import os
HOME = os.getcwd()
print('HOME: ',HOME)

import time
import math
import json
import random
import pandas as pd
import sys
import lxml
import sklearn as sk
import numpy as np

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import HDBSCAN, DBSCAN
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import cosine_similarity

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go

from scipy.sparse import csr_matrix, issparse, lil_matrix

from tqdm import tqdm
from pandarallel import pandarallel

from numba import njit, prange
from numba_progress import ProgressBar
import umap

## CONSTANTS
STANDARD_FILE = 'standard.json'
ACTUAL_FILE = 'actual.json'

OUTPUT_FILE = 'perfectRoute.json'
DATA_DIR = os.path.join(HOME,'data')

K_SHINGLES = 3

HOME:  c:\Users\User\Desktop\VSCodeProjects\DataMining\DataMiningProject23-24


## Loading data from files

In [2]:
# load standard and actual data
with open(os.path.join('data',STANDARD_FILE)) as f:
    standard = json.load(f)

with open(os.path.join('data', ACTUAL_FILE)) as f:
    actual = json.load(f)

# load the data into a dataframe
dfStandard = pd.DataFrame(standard)
dfActual = pd.DataFrame(actual)

In [3]:
# get the unique cities and items of the standard data
cities = []
items = []
drivers = []
longestRoute = 0
shortestRoute = np.inf
maxItemQuantity = 0

standardRefIds = []
for index, s in dfStandard.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    standardRefIds.append(int(idS[1]))
    for trip in route:
        cities.append(trip['from']) 
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
        
    if len(route) < shortestRoute:
        shortestRoute = len(route)

actualRefStandardIds = []
for index, s in dfActual.iterrows():
    #print(s)
    idS = s['id']
    route = s['route']
    idStandard = s['sroute']
    drivers.append(s['driver'])
    actualRefStandardIds.append(int(idStandard[1]))
    for trip in route:
        cities.append(trip['from'])
        items.extend(trip['merchandise'].keys())
        maxItemQuantity = max(maxItemQuantity, max(trip['merchandise'].values()))
        
    if len(route) > 0:
        cities.append(route[-1]['to'])
        
    if len(route) > longestRoute:
        longestRoute = len(route)
    
    if len(route) < shortestRoute:
        shortestRoute = len(route)

# find the unique cities
uniqueCities = sorted(list(set(cities)))
#uniqueCities.insert(0, 'NULL')          # add NULL city, for padding vectors with different lengths (trips in routes)
uniqueItems = sorted(list(set(items)))
# find the unique drivers
uniqueDrivers = list(set(drivers))

if shortestRoute < 2:
    K_SHINGLES = 2

threeShingles = []

for i, c1 in enumerate(uniqueCities):
    for j, c2 in enumerate(uniqueCities):
        if i == j:
            continue
        for k, c3 in enumerate(uniqueCities):
            if j == k or i == k:
                continue
            threeShingles.append([c1, c2, c3])
            
permutations = math.perm(len(uniqueCities), K_SHINGLES)

print("\nUnique cities: ", uniqueCities)
print("Unique drivers: ", uniqueDrivers)
print("Unique items: ", uniqueItems)

print("\nNumber of cities: ", len(uniqueCities))
print("Number of drivers: ", len(uniqueDrivers))
print("Number of items: ", len(uniqueItems))

print("\nMax item quantity: ", maxItemQuantity)

print("\nNumber of three-shingles: ", len(threeShingles))

print(f"\n{K_SHINGLES}-shingles: ", math.perm(len(uniqueCities), K_SHINGLES))
print(f"{K_SHINGLES}-shingles: ", math.comb(len(uniqueCities), K_SHINGLES))

print(f"\n\033[92mK-Shingles used: {K_SHINGLES} \033[0m")


Unique cities:  ['Acerra', 'Benevento', 'Bisceglie', 'Bitonto', 'Bolzano-Bozen', 'Brindisi', 'Caltanissetta', 'Catania', 'Cesena', 'Cinisello Balsamo', 'Como', 'Cuneo', 'Florence', 'Genoa', 'Imola', 'Massa', 'Palermo', 'Pescara', 'Pisa', 'Pistoia', 'Ravenna', 'Reggio Emilia', 'Rovigo', 'Sesto San Giovanni', 'Trapani', 'Udine', 'Verona', 'Vicenza', 'Vigevano', 'Vittoria']
Unique drivers:  ['G', 'I', 'F', 'E', 'C', 'B', 'D', 'A', 'H', 'J']
Unique items:  ['Biscuits', 'Coffee Table', 'Dish Soap', 'Dish Towels', 'First Aid Kit', 'Fitness Tracker', 'Grapes', 'Hat', 'Insect Repellent', 'Laptop', 'Laundry Detergent', 'Mustard', 'Snorkel', 'Soda', 'Strawberries', 'Tackle Box', 'Throw Pillows', 'Tissues', 'Trash Bags', 'Water Bottle']

Number of cities:  30
Number of drivers:  10
Number of items:  20

Max item quantity:  100

Number of three-shingles:  24360

3-shingles:  24360
3-shingles:  4060

K-Shingles used: 3 


## Vectorizing the Routes 

Using one-hot encoding of the all possible tuple of two cities (from-to) and the summed merchandise along a route weighted by the total number of merch

In [4]:
def hashShingles(shingles, n):
    # hash shingles
    string = "" 
    for shingle in shingles:
        string += str(shingle) + "," # [45, 4, 8] -> "45,4,8,"
    
    return hash(string) #% n

def createShingles(df, k, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations):
    # create shingles for each route
    shingles = []
    for index, s in df.iterrows():
        idS = s['id']
        route = s['route']
        shingle = [index]
        citiesInRoute = [] # napoli roma milano teramo bergamo [10,4,5,48,12] [10,4,5] [4,5,48] [5,48,12]
        merchandiseInRoute = np.zeros(len(uniqueItems))
        for trip in route:
            citiesInRoute.append(uniqueCities.index(trip['from']))
            #merchandiseInRoute += np.array(list(trip['merchandise'].values()))
            for item, n in trip['merchandise'].items():
                merchandiseInRoute[uniqueItems.index(item)] += n
        if len(route) > 0:
            citiesInRoute.append(uniqueCities.index(route[-1]['to']))
        if len(route) > 0:
            merchandiseInRoute = merchandiseInRoute / (maxItemQuantity*len(route))
        
        hashedShingles = []
        for i in range(len(citiesInRoute)-k+1):
            hashedShingles.append(hashShingles(citiesInRoute[i:i+k], permutations) )
        
        shingle.append(np.array(hashedShingles))
        
        shingle.append(merchandiseInRoute) # quantity hot encoding
        
        shingles.append(shingle)
        
    return shingles # [ index, [shingles], [merchandise] ]

def create_shingles(s, k, uniqueCities, uniqueItems, longestRoute, maxItemQuantity, permutations):
    idS = s['id']
    route = s['route']
    shingle = [s.name]
    citiesInRoute = [] 
    merchandiseInRoute = np.zeros(len(uniqueItems))
    for trip in route:
        citiesInRoute.append(uniqueCities.index(trip['from']))
        for item, n in trip['merchandise'].items():
            merchandiseInRoute[uniqueItems.index(item)] += n
    if len(route) > 0:
        citiesInRoute.append(uniqueCities.index(route[-1]['to']))
    if len(route) > 0:
        merchandiseInRoute = merchandiseInRoute / (maxItemQuantity*len(route))
    
    hashedShingles = []
    for i in range(len(citiesInRoute)-k+1):
        hashedShingles.append(hashShingles(citiesInRoute[i:i+k], permutations))
    
    shingle.append(np.array(hashedShingles))
    shingle.append(merchandiseInRoute)
    
    return shingle

In [5]:
#standardSets = createShingles(dfStandard, k=K_SHINGLES, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)
#actualSets = createShingles(dfActual, k=K_SHINGLES, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)

#print("\nstandardSets:", len(standardSets))
#print("actualSets:", len(actualSets))

#assert len(standardSets[0]) == 3, "The length of the standard set is not equal to 3 (index, shingles, merchandise)"
#assert len(standardSets[0][2]) == len(uniqueItems), "The length of the merchandise vector is not equal to the number of unique items"


standardSets: 20
actualSets: 940


In [5]:
## Dictionary containing all the Sets [idx, [shingles], [quantity-hot merch]] for all the drivers
actualDriverSets = {}
for driver in uniqueDrivers:
    dfActualDriver = dfActual[dfActual['driver'] == driver]
    actualDriverSets[driver] = createShingles(dfActualDriver, k=K_SHINGLES, uniqueCities=uniqueCities, uniqueItems=uniqueItems, longestRoute=longestRoute, maxItemQuantity=maxItemQuantity, permutations=permutations)

## Compute Route Similarity Matrix

In [8]:
def jaccard_similarity_matrix(matrix):
    if 1.0 - np.count_nonzero(matrix) / matrix.size > 0.5:
        #print("matrix jaccard is sparse")
        
        matrixCSR = csr_matrix(matrix)
        intersection = np.dot(matrixCSR, matrixCSR.T)
        intersection = intersection.todense()
        #print("intersection", intersection.shape, type(intersection))
        #print("intersection", intersection.toarray(), type(intersection.toarray()))
        row_sums = matrix.sum(axis=1)
        #print("row_sums", row_sums.shape)
        union = row_sums[:, None] + row_sums - intersection
        #print("union", union.shape)
        union = np.where(union == 0, 1, union)  # avoid division by zero
        #print("union", union.shape)
        jaccard_similarity = intersection / union
        #print("jaccard_similarity", jaccard_similarity.shape, type(jaccard_similarity))
    else:
        #print("matrix jaccard is not sparse")
        
        intersection = np.dot(matrix, matrix.T)
        #print("intersection", intersection.shape)
        #print("intersection", intersection.toarray(), type(intersection.toarray()))
        row_sums = matrix.sum(axis=1)
        #print("row_sums", row_sums.shape)
        union = row_sums[:, None] + row_sums - intersection
        #print("union", union.shape)
        union = np.where(union == 0, 1, union)
        jaccard_similarity = intersection / union
        #print("jaccard_similarity", jaccard_similarity.shape)
    #print("jaccard_similarity contains nan", np.isnan(jaccard_similarity).any())   
    return jaccard_similarity

def jaccard_similarity_minhash(matrix):
    similarity_matrix = np.full((matrix.shape[0], matrix.shape[0]), np.inf)
    num_permutations = matrix.shape[1]
    for i in tqdm(range(matrix.shape[0])):
        for j in range(i + 1, matrix.shape[0]):
            similarity_matrix[i, j] = np.count_nonzero(matrix[i, :] == matrix[j, :]) / num_permutations
            similarity_matrix[j, i] = similarity_matrix[i, j]
                    
    # Create a full symmetric matrix from the upper triangular part
    #similarity_matrix = np.triu(similarity_matrix) + np.triu(similarity_matrix, 1).T
    np.fill_diagonal(similarity_matrix, 1)
    #print("similarity_matrix", similarity_matrix.shape, similarity_matrix[0])
    return similarity_matrix


def hash_function_hash_code(num_of_hashes,n_col,next_prime):
  
    #coeffA = np.array(pick_random_coefficients(num_of_hashes,max_column_length)).reshape((num_of_hashes,1))
    #coeffB = np.array(pick_random_coefficients(num_of_hashes,max_column_length)).reshape((num_of_hashes,1))

    coeffA = np.array(random.sample(range(0,n_col*100),num_of_hashes)).reshape((num_of_hashes,1))
    coeffB = np.array(random.sample(range(0,n_col*100),num_of_hashes)).reshape((num_of_hashes,1))

    x = np.arange(n_col).reshape((1,n_col))

    hash_code = (np.matmul(coeffA,x) + coeffB) % next_prime # (num_of_hashes,n_col) so how each column index is permuted

    return hash_code

def minhash(u,num_of_hashes):
    (n_row, n_col) = u.shape
    next_prime = n_col
    hash_code = hash_function_hash_code(num_of_hashes,n_col,next_prime)

    signature_array = np.empty(shape = (n_row,num_of_hashes))

    #t2 = time.time()

    for row in tqdm(range(n_row), desc="minhashing"):
        #print("row", row)
        ones_index = np.where(u[row,:]==1)[0]
        #if len(ones_index) == 0:
        signature_array[row,:] = np.zeros((1,num_of_hashes))
            #continue
        corresponding_hashes = hash_code[:,ones_index]
        #print("ones_index", ones_index.shape, ones_index)
        #print("corresponding_hashes", corresponding_hashes.shape, corresponding_hashes)
        row_signature = np.amin(corresponding_hashes,axis=1).reshape((1,num_of_hashes))

        signature_array[row,:] = row_signature

    return signature_array

def find_band_and_row_values(columns, threshold):
    previous_b = 1
    previous_r = columns
    for b in range(1, columns + 1):
        if columns % b == 0:
            r = columns // b
            if (1 / b) ** (1 / r)  <= threshold:
                if np.abs((1 / previous_b) ** (1 / previous_r) - threshold) < np.abs((1 / b) ** (1 / r) - threshold):
                    return previous_b, previous_r
                return b, r
    return columns, 1

def lsh(minhash_matrix, thresh_user=0.2):
    # Initialize the signature matrix
    columns = minhash_matrix.shape[1]
    
    # Generate the hash functions
   # hash_functions = [lambda x, a=a, b=b: (a * x + b) % minhash_matrix.shape[1] for a, b in zip(random.sample(range(1000), bands), random.sample(range(1000), bands))]
    hash_function = lambda x: hash(",".join([str(x[i]) for i in range(len(x))]))
    
    # b = bands
    # r = columns//bands
    b, r = find_band_and_row_values(columns, thresh_user)
    # If columns is not divisible by bands
    if columns % b != 0:
        # Find the closest number that makes it divisible
        while columns % b != 0:
            b -= 1
        r = columns // b
    #bands = b
        
    #print("final bands", b)
    signature_matrix = np.full((minhash_matrix.shape[0], b), np.inf)
    
    # if threshold is 0.8,
    threshold = (1 / b) ** (1 / r) 
    #print("lsh threshold", threshold)
    
    # For each band
    #print("Computing hash values of bands...")
    hash_values = np.apply_along_axis(lambda x: hash_function(x) % minhash_matrix.shape[0], 1, minhash_matrix.reshape(-1, r))
    # Reshape the hash values to match the signature matrix
    hash_values = hash_values.reshape(minhash_matrix.shape[0], b)
    # Update the signature matrix
    signature_matrix = hash_values
            
    # find candidate pairs
    #print("Finding candidate pairs...")
    candidate_pairs = []
    for i in tqdm(range(signature_matrix.shape[0])):
        # Compute the similarity of the current row with all following rows
        similarities = np.sum(signature_matrix[i+1:, :] == signature_matrix[i, :], axis=1) / b
        # Find the indices of the rows that have a similarity greater than or equal to the threshold
        indices = np.nonzero(similarities >= threshold)[0]
        # Add the pairs to the candidate pairs
        candidate_pairs.extend((i, i+1+index) for index in indices)
    
    return candidate_pairs

def lsh_two_matrices(minhash_matrix1, minhash_matrix2, thresh_user=0.2):
    # Initialize the signature matrix
    columns = minhash_matrix1.shape[1]
    
    # Generate the hash functions
    # hash_functions = [lambda x, a=a, b=b: (a * x + b) % minhash_matrix.shape[1] for a, b in zip(random.sample(range(1000), bands), random.sample(range(1000), bands))]
    hash_function = lambda x: hash(",".join([str(x[i]) for i in range(len(x))]))
    
    # b = bands
    # r = columns//bands
    b, r = find_band_and_row_values(columns, thresh_user)
    # If columns is not divisible by bands
    if columns % b != 0:
        # Find the closest number that makes it divisible
        while columns % b != 0:
            b -= 1
        r = columns // b
    #bands = b
        
    #print("final bands", b)
    signature_matrix1 = np.full((minhash_matrix1.shape[0], b), np.inf)
    signature_matrix2 = np.full((minhash_matrix2.shape[0], b), np.inf)
    
    # if threshold is 0.8,
    threshold = (1 / b) ** (1 / r) 
    #print("lsh threshold", threshold)
    
    # For each band
    #print("Computing hash values of bands...")
    hash_values1 = np.apply_along_axis(lambda x: hash_function(x) % minhash_matrix1.shape[0], 1, minhash_matrix1.reshape(-1, r))
    hash_values2 = np.apply_along_axis(lambda x: hash_function(x) % minhash_matrix2.shape[0], 1, minhash_matrix2.reshape(-1, r))
    # Reshape the hash values to match the signature matrix
    hash_values1 = hash_values1.reshape(minhash_matrix1.shape[0], b)
    hash_values2 = hash_values2.reshape(minhash_matrix2.shape[0], b)
    # Update the signature matrix
    signature_matrix1 = hash_values1
    signature_matrix2 = hash_values2
            
    # find candidate pairs
    #print("Finding candidate pairs...")
    candidate_pairs = np.empty((minhash_matrix1.shape[0], 2))
    for i in tqdm(range(signature_matrix1.shape[0])):
        # Compute the similarity of the current row with all following rows
        similarities = np.sum(signature_matrix2 == signature_matrix1[i, :], axis=1) / b
        # Find the indices of the rows that have a similarity greater than or equal to the threshold
        #indices = np.nonzero(similarities >= threshold)[0]
        indexMax = np.argmax(similarities)
        simMax = similarities[indexMax]
        # Add the pairs to the candidate pairs
        #candidate_pairs.extend((i, i+1+index) for index in indices)
        candidate_pairs[i] = [indexMax, simMax]
        
    return candidate_pairs
        


@njit(cache=True,)
def replicate_row(matrix, i):
    result = np.empty((matrix.shape[0], matrix.shape[1]))
    for j in range(matrix.shape[0]):
        result[j] = matrix[i]
    return result

@njit(cache=True, nogil=True, parallel=True)
def compute_subset_similarity_matrix(subset_matrix, progress_proxy):
    n = subset_matrix.shape[0]
    m = subset_matrix.shape[1]
    subset_matrix = subset_matrix.astype(np.int64)
    subset_similarity_matrix = np.zeros((n, n))
    subset2 = subset_matrix
    for i in prange(n):
        subset1 = subset_matrix[i].reshape(1, -1) #replicate_row(subset_matrix, i)    
        min_matrix = np.minimum(subset1, subset2)
        sum_min_matrix = np.sum(min_matrix, axis=-1)
        
        max_matrix = np.maximum(subset1, subset2)
        sum_max_matrix = np.sum(max_matrix, axis=-1)
        
        subset_similarity_matrix[i] = (np.divide(sum_min_matrix, sum_max_matrix)).T
        progress_proxy.update(1)
    return subset_similarity_matrix


def jaccard_similarity_minhash_lsh(matrix, thresh_user=0.2):
    #similarity_matrix = csr_matrix((matrix.shape[0], matrix.shape[0]), dtype=np.float64)
    similarity_matrix = lil_matrix((matrix.shape[0], matrix.shape[0]), dtype=np.float64)
    pairs = lsh(matrix, thresh_user=thresh_user)
    #uniqueRows = np.unique([i for i, j in pairs] + [j for i, j in pairs])
    uniqueRowsSet = set([i for i, j in pairs] + [j for i, j in pairs])
    #print("uniqueRows numpy", len(uniqueRows))
    #print("num of subset of rows to check similarity:", len(uniqueRowsSet))
    #print(" num of pairs", len(uniqueRowsSet)*(len(uniqueRowsSet)-1)/2)
    #print(" instead of", matrix.shape[0]*(matrix.shape[0]-1)/2)
    #print("improved by", (1-len(uniqueRowsSet)*(len(uniqueRowsSet)-1)/2/(matrix.shape[0]*(matrix.shape[0]-1)/2))*100, "%")
    #print("num of pairs", len(pairs))
    #print("num unique i", len(set([i for i, j in pairs])))
    #print("num unique j", len(set([j for i, j in pairs])))
    #print("num unique rows", len(uniqueRows))
    map_i = {i: index for i, index in enumerate(uniqueRowsSet)}
    map_i_array = np.array([map_i[i] for i in range(len(map_i))])
    
    subset_matrix = matrix[list(uniqueRowsSet)]
    
    subset_similarity_matrix = np.full((subset_matrix.shape[0], subset_matrix.shape[0]), np.inf)
    
    #print("Computing jaccard similarity on subset matrix...")
    #print("subset matrix", subset_matrix.shape)
    
    with ProgressBar(total=subset_matrix.shape[0]) as progress:
        subset_similarity_matrix = compute_subset_similarity_matrix(subset_matrix, progress)
    
    # map back to original matrix
    #print("Mapping back to original matrix...")
    # Create arrays of indices
    indices_i, indices_j = np.triu_indices(subset_similarity_matrix.shape[0], k=1)
    similarity_matrix = similarity_matrix.tocsr()
    # Update the similarity matrix
    similarity_matrix[map_i_array[indices_i], map_i_array[indices_j]] = subset_similarity_matrix[indices_i, indices_j]
    similarity_matrix[map_i_array[indices_j], map_i_array[indices_i]] = subset_similarity_matrix[indices_i, indices_j]
    
    similarity_matrix.setdiag(1)
    
    return similarity_matrix

def jaccard_similarity_minhash_lsh_two_matrices(matrix1, matrix2, thresh_user=0.2):
    #similarity_matrix = csr_matrix((matrix.shape[0], matrix.shape[0]), dtype=np.float64)
    similarity_matrix = lil_matrix((matrix1.shape[0], matrix2.shape[0]), dtype=np.float64)
    pairs = lsh(matrix1, thresh_user=thresh_user)
    #uniqueRows = np.unique([i for i, j in pairs] + [j for i, j in pairs])
    uniqueRowsSet = set([i for i, j in pairs] + [j for i, j in pairs])
    #print("uniqueRows numpy", len(uniqueRows))
    #print("num of subset of rows to check similarity:", len(uniqueRowsSet))
    #print(" num of pairs", len(uniqueRowsSet)*(len(uniqueRowsSet)-1)/2)
    #print(" instead of", matrix1.shape[0]*(matrix1.shape[0]-1)/2)
    #print("improved by", (1-len(uniqueRowsSet)*(len(uniqueRowsSet)-1)/2/(matrix1.shape[0]*(matrix1.shape[0]-1)/2))*100, "%")
    #print("num of pairs", len(pairs))
    #print("num unique i", len(set([i for i, j in pairs])))
    #print("num unique j", len(set([j for i, j in pairs])))
    #print("num unique rows", len(uniqueRows))
    map_i = {i: index for i, index in enumerate(uniqueRowsSet)}
    map_i_array = np.array([map_i[i] for i in range(len(map_i))])
    
    subset_matrix1 = matrix1[list(uniqueRowsSet)]
    subset_matrix2 = matrix2[list(uniqueRowsSet)]
    
    subset_similarity_matrix = np.full((subset_matrix1.shape[0], subset_matrix2.shape[0]), np.inf)
    
    #print("Computing jaccard similarity on subset matrix...")
    #print("subset matrix", subset_matrix1.shape)
    

def jaccard_similarity_two_matrices(matrix1, matrix2):
    #intersection = np.dot(matrix, matrix.T)
    intersection = np.dot(matrix1, matrix2.T)
    row_sums1 = matrix1.sum(axis=1)
    row_sums2 = matrix2.sum(axis=1)
    union = row_sums1[:, None] + row_sums2 - intersection
    union = np.where(union == 0, 1, union)  # avoid division by zero
    jaccard_similarity = intersection / union
    return jaccard_similarity

def jaccard_similarity_matrix_merch(matrix):
    #print("matrix", matrix.shape)
    min_matrix = np.minimum(matrix[:, None, :], matrix[None, :, :]) # (10, 100) -> (10, 1, 100) -> (1, 10, 100) -> (10, 10, 100)
    sum_min_matrix = np.sum(min_matrix, axis=-1)
    #print("sum_min_matrix", sum_min_matrix.shape)
    
    max_matrix = np.maximum(matrix[:, None, :], matrix[None, :, :])
    sum_max_matrix = np.sum(max_matrix, axis=-1)
    #print("sum_max_matrix", sum_max_matrix.shape)
    
    jaccard_similarity = sum_min_matrix / sum_max_matrix
    return jaccard_similarity
    
def similarity_matrix_merch(matrix):
    if 1.0 - np.count_nonzero(matrix) / matrix.size > 0.5:
        #print("merch matrix is sparse")
        matrix = csr_matrix(matrix)
    #print("matrix merch shape", matrix.shape)
    simMatrix = cosine_similarity(matrix)
    
    return simMatrix

def create_binary_matrix(routeSets):
    uniqueShingles = list(set(shingle for route in routeSets for shingle in route[1]))
    #print("uniqueShingles", len(uniqueShingles))

    # Create a dictionary that maps each shingle to its index
    shingle_to_index = {shingle: index for index, shingle in enumerate(uniqueShingles)}
    #print("shingle_to_index", len(shingle_to_index))

    binaryMatrix = np.zeros((len(routeSets), len(uniqueShingles)), dtype=int)

    for i, route in enumerate(routeSets):
        #print("i", i)
        # Get the indices of the shingles in this route
        indices = [shingle_to_index[shingle] for shingle in route[1]]
        # Use advanced indexing to set the corresponding elements in the binary matrix to 1
        binaryMatrix[i, indices] = 1

    return binaryMatrix

def create_binary_matrix_minhash(matrix):
    numUnique = np.unique(matrix)
    binaryMatrix = np.zeros((matrix.shape[0], len(numUnique)), dtype=int)
    for i, route in enumerate(matrix):
        indices = np.where(route == 1)[0]
        binaryMatrix[i, indices] = 1
    
    return binaryMatrix

def create_binary_matrices(routeSet1, routeSet2):
    # create binary matrix where each row represents a route
    uniqueShinglesBoth = list(set([shingle for route in routeSet1 for shingle in route[1]] + [shingle for route in routeSet2 for shingle in route[1]]))
    binaryMatrix1 = np.zeros((len(routeSet1), len(uniqueShinglesBoth)))
    binaryMatrix2 = np.zeros((len(routeSet2), len(uniqueShinglesBoth)))
    for i, route in enumerate(routeSet1):
        for shingle in route[1]:
            binaryMatrix1[i][uniqueShinglesBoth.index(shingle)] = 1
            
    for i, route in enumerate(routeSet2):
        for shingle in route[1]:
            binaryMatrix2[i][uniqueShinglesBoth.index(shingle)] = 1
    return binaryMatrix1, binaryMatrix2

def find_num_hashes_minhash(matrix):
    if matrix.shape[1] < 1000:
        num_hash_functions = matrix.shape[1]//10
    elif matrix.shape[1] < 10_000:
        num_hash_functions = 150
    elif matrix.shape[1] < 100_000:
        num_hash_functions = 250
    else:
        num_hash_functions = 300
    return num_hash_functions

def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def makeSymmetric(mat):
    for i in range(0, mat.shape[0]):
        for j in range(0, mat.shape[1]):
            if (j < i):
                mat[i][j] = mat[j][i]


In [9]:
## Dictionary containing the distance matrix for all the drivers
driversDistances = {}
for driver, driverSet in actualDriverSets.items():
    route_matrix = create_binary_matrix(driverSet)

    num_hash_functions = find_num_hashes_minhash(route_matrix)
    route_matrix = minhash(route_matrix, num_hash_functions if num_hash_functions % 2 == 0 else num_hash_functions + 1)

    merch_matrix = np.array([s[2] for s in driverSet])

    route_similarity = jaccard_similarity_minhash_lsh(route_matrix, thresh_user=0.4)
    merch_similarity = similarity_matrix_merch(merch_matrix)

    actualSetDistances = route_similarity * merch_similarity
    actualSetDistances = np.nan_to_num(actualSetDistances, nan=0)
    actualSetDistances = 1 - actualSetDistances

    if not check_symmetric(actualSetDistances):
        makeSymmetric(actualSetDistances)

    driversDistances[driver] = actualSetDistances
    

100%|██████████| 97/97 [00:00<00:00, 48578.80it/s]


0it [00:00, ?it/s]

c:\Users\User\Desktop\VSCodeProjects\DataMining\DataMiningProject23-24\datamining\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
100%|██████████| 100/100 [00:00<00:00, 50021.51it/s]


  0%|          | 0/21 [00:00<?, ?it/s]

c:\Users\User\Desktop\VSCodeProjects\DataMining\DataMiningProject23-24\datamining\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
100%|██████████| 102/102 [00:00<00:00, 25507.93it/s]


...

c:\Users\User\Desktop\VSCodeProjects\DataMining\DataMiningProject23-24\datamining\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
100%|██████████| 82/82 [00:00<00:00, 41051.91it/s]


  0%|          | 0/15 [00:00<?, ?it/s]

c:\Users\User\Desktop\VSCodeProjects\DataMining\DataMiningProject23-24\datamining\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
100%|██████████| 88/88 [00:00<00:00, 43961.26it/s]


  0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\User\Desktop\VSCodeProjects\DataMining\DataMiningProject23-24\datamining\Lib\site-packages\scipy\sparse\_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [38]:
driversDistances['A'][0]

array([1.11022302e-16, 5.92918950e-02, 6.95212928e-02, 8.18171151e-02,
       6.47919280e-02, 8.31293478e-02, 4.74533465e-02, 4.97502003e-02,
       1.36907115e-01, 5.24399704e-02, 6.94066774e-02, 8.20866750e-02,
       8.48821230e-02, 5.82876515e-02, 1.59967836e-01, 1.85686625e-01,
       6.88246448e-02, 7.21612606e-02, 2.58959630e-02, 9.50998972e-02,
       1.40986598e-01, 8.52692249e-02, 2.58691505e-02, 5.26999422e-02,
       6.61104655e-02, 2.12193148e-01, 1.77091666e-01, 1.54768809e-01,
       7.40508430e-02, 9.93661689e-02, 7.15885512e-02, 4.94263017e-02,
       5.75970624e-02, 6.47370700e-02, 5.92693896e-02, 6.80977622e-02,
       5.23468789e-02, 9.70279049e-02, 9.57436831e-02, 6.23264359e-02,
       7.20805082e-02, 2.20054838e-02, 7.92483858e-02, 1.23670695e-01,
       1.21791287e-01, 6.24001199e-02, 5.71004425e-02, 9.81162417e-02,
       5.62745468e-02, 4.44540517e-02, 2.03642108e-01, 8.43551738e-02,
       6.91573081e-02, 1.39098839e-01, 3.64018139e-02, 6.76984169e-02,
      

## HDBSCAN

In [37]:
from sklearn.metrics import pairwise_distances
forward_expansion = len(dfActual) // len(dfStandard)

driversClusters = {}
for driver, driverDistance in driversDistances.items():
    if driver != 'A':
        continue
    print('performing DBSCAN for driver ', driver)

    actualDistance = np.array(driverDistance)
    print(len(actualDistance))
    best_min = 0
    best_max = 0
    best_n_cluster=np.inf
    for min_cluster in range(2, forward_expansion, 2):
        for max_cluster in range(forward_expansion, forward_expansion*2, 2):
            hdb = HDBSCAN(min_cluster_size=min_cluster, max_cluster_size=max_cluster, metric="precomputed", store_centers=None,allow_single_cluster=False ).fit(actualDistance.copy())
            noise_count = sum([1 for noise in hdb.labels_ if noise < 0])
            if noise_count < best_n_cluster:
                best_n_cluster = noise_count
                best_min = min_cluster
                best_max = max_cluster

    print(f'best min {best_min} best max {best_max}')
    hdb = HDBSCAN(min_cluster_size=best_min, max_cluster_size=len(actualDistance)//20, metric="precomputed", store_centers=None,allow_single_cluster=False ).fit(actualDistance.copy())
    print(f"num clusters found for driver {driver}: {len(set(hdb.labels_))}")
    print('labels: ', hdb.labels_)
    labels = hdb.labels_

    ##Plotting
    #perplexity = 50 if len(driversDistances[driver]) > 50 else len(driversDistances[driver]) - 1
    driversTSNE = TSNE(n_components=3, perplexity=5, n_iter=1000, verbose=1).fit_transform(driversDistances[driver])

    colors = plt.cm.jet(np.linspace(0, 1, len(set(labels))))
    color_map = dict(zip(set(labels), colors))
    marker_colors = [color_map[label] if label > -1 else np.array([0,0,0,1]) for label in labels]
    traceStandard = go.Scatter3d(
    x=driversTSNE[:,0],
    y=driversTSNE[:,1],
    z=driversTSNE[:,2],
    mode='markers',
    marker=dict(
        size=7,
        color=marker_colors,                # set color to an array/list of desired values
        opacity=1,
        symbol='diamond'
    )
    )

    # Plot
    data = [traceStandard]

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=data, layout=layout)
    fig.show()

    ## Selecting the cluster that has the higher intra-cluster similarity
    # Compute pairwise similarity/distance matrix within clusters
    pairwise_similarities = []
    for label in np.unique(labels):
        cluster_points = actualDistance[labels == label]
        pairwise_similarity = 1 - pairwise_distances(cluster_points, metric='cosine')
        pairwise_similarities.append(pairwise_similarity)

    # Compute the average similarity within each cluster
    avg_similarities = [np.mean(similarity) for similarity in pairwise_similarities]

    # Select the cluster with the highest average similarity
    selected_cluster = np.argmax(avg_similarities)

    # Access the data points in the selected cluster
    selected_cluster_points = actualDistance[labels == selected_cluster]

    driversClusters[driver] = selected_cluster_points
    break

performing DBSCAN for driver  A
83
best min 2 best max 47
num clusters found for driver A: 2
labels:  [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0
 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1]
[t-SNE] Computing 16 nearest neighbors...
[t-SNE] Indexed 83 samples in 0.000s...
[t-SNE] Computed neighbors for 83 samples in 0.011s...
[t-SNE] Computed conditional probabilities for sample 83 / 83
[t-SNE] Mean sigma: 3.939326
[t-SNE] KL divergence after 250 iterations with early exaggeration: 101.782616
[t-SNE] KL divergence after 1000 iterations: 0.870496


In [28]:
completeSets = actualSets + standardSets
print("completeSets: ", len(completeSets))
route_matrix_with_standard = create_binary_matrix(completeSets)
# binary matrix where each row represents merchandise
merch_matrix_with_standard = np.array([s[2] for s in completeSets])

# compute Jaccard similarity for each matrix
route_similarity_with_standard = jaccard_similarity_matrix(route_matrix_with_standard)
merch_similarity_with_standard = similarity_matrix_merch(merch_matrix_with_standard)
completeSetsDistances = (route_similarity_with_standard + merch_similarity_with_standard) / 2
completeSetsDistances = np.nan_to_num(completeSetsDistances, nan=0)
completeSetsDistances = 1 - completeSetsDistances
completeSetsDistances = np.array(completeSetsDistances)
print("completeSetsDistances", completeSetsDistances.shape, completeSetsDistances[0])

perplexity = 50 if len(completeSetsDistances) > 50 else len(completeSetsDistances) - 1
completeSetTSNE = TSNE(n_components=3, perplexity=perplexity, n_iter=1000, verbose=1).fit_transform(completeSetsDistances)

completeSets:  960
uniqueShingles 4876
shingle_to_index 4876
matrix jaccard is sparse
intersection (960, 960) <class 'numpy.matrix'>
row_sums (960,)
union (960, 960)
union (960, 960)
jaccard_similarity (960, 960) <class 'numpy.matrix'>
jaccard_similarity contains nan False
matrix merch shape (960, 20)
completeSetsDistances (960, 960) [0.         0.55077636 0.57453439 0.5453838  0.56458047 0.55220908
 0.58563433 0.56650395 0.54509347 0.5366262  0.57842986 0.54941341
 0.54728318 0.53789171 0.54430558 0.55221679 0.54051946 0.52743088
 0.52954894 0.53912212 0.55389263 0.53563029 0.54419849 0.55263574
 0.57190877 0.54163961 0.54962128 0.53406769 0.55396504 0.54021546
 0.55136272 0.53349924 0.58155198 0.34738772 0.56075361 0.54847454
 0.55144335 0.5554357  0.55351051 0.56456724 0.53709901 0.56927187
 0.56662505 0.57092426 0.63316747 0.55443415 0.56562964 0.55416709
 0.54993827 0.52633399 0.57517867 0.53287707 0.56269798 0.54571827
 0.53620079 0.57959857 0.55622482 0.54245733 0.54891751 0.528

In [ ]:
forward_expansion = len(dfActual) // len(uniqueDrivers)

for distance in actualDistances:
    actualDistance = np.array(distance)

    hdb = HDBSCAN(min_cluster_size=2, max_cluster_size=forward_expansion*2, metric="precomputed", store_centers=None,allow_single_cluster=False ).fit(actualDistance)

    labels_HDBSCAN = hdb.labels_
    unique_labels = np.unique(labels_HDBSCAN)

    medoidsIndices = []
    cluster_mean_distances = []
    for cluster in unique_labels:
        if cluster in [-1, -2, -3]:
            continue
        cluster_elements = np.where(labels_HDBSCAN == cluster)[0]
        cluster_distances = actualSetsDistances[cluster_elements][:,cluster_elements]
        cluster_distances_sum = np.sum(cluster_distances, axis=1)
        cluster_distances_mean = np.mean(cluster_distances, axis=1)
        cluster_mean_distances.append(np.min(cluster_distances_mean))
        medoid = cluster_elements[np.argmin(cluster_distances_sum)]
        medoidsIndices.append(medoid)
    medoidsIndices = np.array(medoidsIndices)

## 3D Visualization

In [29]:
# reorder the labels to have colors matching the cluster results, using medoids which are closer to the standard vectors
medoidSets = [actualSets[i] for i in medoidsIndices]
print("medoidSets: ", len(medoidSets))

num_clusters_unique = unique_labels[unique_labels >= 0]
#print("num_clusters_unique: ", len(num_clusters_unique), num_clusters_unique)

assert len(medoidSets) == len(num_clusters_unique), "The number of medoids is not equal to the number of unique labels"   

if len(medoidSets) == 0:
    print("No clustroids found")
else:

    route_matrix_standard, route_matrix_medoids = create_binary_matrices(standardSets, medoidSets)

    simMatrixMixed = jaccard_similarity_two_matrices(route_matrix_medoids, route_matrix_standard)

    CAN_BE_ORDERED = False
    # get the closest standard vector for each medoid using simMatrixMixed

    argmax = np.argmax(simMatrixMixed, axis=1) # get the index of the closest standard vector for each medoid
    print("argmax: ", argmax.shape, argmax)

    if len(set(argmax)) == len(medoidsIndices): # if the argmax are all different, then the medoids can be reordered
        print("argmax is correct, can be reordered")
        CAN_BE_ORDERED = True

        unique_labels_reordered = argmax   # 4, 2, 5, 10, ... -> 10, 4, 5, 2, ...


    if not CAN_BE_ORDERED:

        print("unique_labels: ", len(unique_labels), unique_labels)
    else:
        print("unique_labels_reordered: ", len(unique_labels_reordered), unique_labels_reordered)

        
    distancesClustroids = []
    distancesStandardVectors = []
    distancesStandardVectors2 = []

    actualRefStandardIdsNumpy = np.array(actualRefStandardIds)
    for i, stdID in enumerate(standardRefIds):
        distStdCluster = completeSetsDistances[len(actualSets):, :len(actualSets)][i, np.where(actualRefStandardIdsNumpy == stdID)[0]]
        #print("distStdCluster: ", np.mean(distStdCluster), len(distStdCluster), distStdCluster)
        distancesStandardVectors.append(np.mean(distStdCluster))
    
    mean_distance_clustroids = np.mean(cluster_mean_distances)
    std_dev_distance_clustroids = np.std(cluster_mean_distances)

    mean_distance_standard_vectors = np.mean(distancesStandardVectors)
    std_dev_distance_standard_vectors = np.std(distancesStandardVectors)

    cv_clustroids = std_dev_distance_clustroids / mean_distance_clustroids
    cv_standard_vectors = std_dev_distance_standard_vectors / mean_distance_standard_vectors
    # print in green if the improvement is positive, in red if it is negative
    print("\n\033[94mMean distance from vectors of the same cluster to:")
    print("         clustroids: ", mean_distance_clustroids)
    #print("         first     : ", np.mean(distancesClustroids))
    print("   standard vectors: ", mean_distance_standard_vectors)
    #print("first                ", np.mean(distancesStandardVectors2))
    

    print("\nStd dev distance from vectors of the same cluster to:")
    print("         clustroids: ", std_dev_distance_clustroids)
    print("   standard vectors: ", std_dev_distance_standard_vectors)

    print("\nCoefficient of variation from vectors of the same cluster to:")
    print("         clustroids: ", cv_clustroids)
    print("   standard vectors: ", cv_standard_vectors)
    print("\033[0m")

    #ratio = np.mean(distancesClustroids) / np.mean(distancesStandardVectors)
    ratio = mean_distance_clustroids / mean_distance_standard_vectors
    percentage = (1 - ratio) * 100

    print("\033[93mMean:\033[0m")
    if percentage > 0:
        # print in green if the improvement is positive, in red if it is negative
        print("   Improvement: \033[92m{:.2f}% \033[0m".format(percentage))
    else:
        print("   Decline: \033[91m{:.2f}% \033[0m".format(-percentage))
        
    
    ratio = std_dev_distance_clustroids / std_dev_distance_standard_vectors
    percentage = (1 - ratio) * 100
    print("\033[93m\nStd dev:\033[0m")
    if percentage > 0:
        # print in green if the improvement is positive, in red if it is negative
        print("   Improvement: \033[92m{:.2f}% \033[0m".format(percentage))
    else:
        print("   Decline: \033[91m{:.2f}% \033[0m".format(-percentage))
        
    ratio = cv_clustroids / cv_standard_vectors
    percentage = (1 - ratio) * 100
    print("\033[93m\nTotal\033[0m")
    if percentage > 0:
        # print in green if the improvement is positive, in red if it is negative
        print("   Improvement: \033[92m{:.2f}% \033[0m".format(percentage))
    else:
        print("   Decline: \033[91m{:.2f}% \033[0m".format(-percentage))

medoidSets:  4
argmax:  (4,) [18 15  5  5]
unique_labels:  5 [-1  0  1  2  3]

Mean distance from vectors of the same cluster to:
         clustroids:  -31.6447681004572
   standard vectors:  0.44954556734819534

Std dev distance from vectors of the same cluster to:
         clustroids:  18.365654282101783
   standard vectors:  0.0916047813359171

Coefficient of variation from vectors of the same cluster to:
         clustroids:  -0.5803693749247744
   standard vectors:  0.20377195992895789

Mean:
   Improvement: 7139.28% 

Std dev:
   Decline: 19948.79% 

Total
   Improvement: 384.81% 


In [30]:
max_len = max(len(standardSets), len(num_clusters_unique))
colors = plt.cm.jet(np.linspace(0, 1, max_len))
# print("colors", colors.shape, colors)

# print("medoid labels", [labels_HDBSCAN[i] for i in medoidsIndices])
# print("argmax", argmax)

if len(medoidSets) > 0 and CAN_BE_ORDERED:
    print("medoids added to plots and reordered")
    color_map = dict(zip(range(max_len), colors[unique_labels_reordered]))
    #rint("color_map", color_map)
else:
    color_map = dict(zip(range(max_len), colors))   # 0=red, 1=blue, 2=green, 3=yellow, 4=purple, 5=lightblue, 6=lightgreen, 7=lightyellow, 8=lightpurple
    
marker_colors = [color_map[label] if label > -1 else np.array([0,0,0,1]) for label in labels_HDBSCAN]
marker_colors_medoids = [color_map[label] if label > -1 else np.array([0,0,0,1]) for label in labels_HDBSCAN[medoidsIndices]]
#marker_colors_medoids = [color_map[label] if label > -1 else np.array([0,0,0,1]) for label in unique_labels]


# Create a trace for each type (centroids data)
traceStandard = go.Scatter3d(
    x=completeSetTSNE[len(actualSets):,0],
    y=completeSetTSNE[len(actualSets):,1],
    z=completeSetTSNE[len(actualSets):,2],
    mode='markers',
    marker=dict(
        size=7,
        color=colors,                # set color to an array/list of desired values
        opacity=1,
        symbol='diamond'
    )
)

if len(medoidSets) > 0:
    medoidsElements = completeSetTSNE[medoidsIndices] # medoidsIndices = [921, 123]
    traceMedoids = go.Scatter3d(
        x=medoidsElements[:,0],
        y=medoidsElements[:,1],
        z=medoidsElements[:,2],
        mode='markers',
        marker=dict(
            size=7,
            color=marker_colors_medoids,                # set color to an array/list of desired values
            opacity=1,
            symbol='cross'
        )
    )

# Create a trace for each type (centroids data)
traceActual = go.Scatter3d(
    x=completeSetTSNE[:len(actualSets),0],
    y=completeSetTSNE[:len(actualSets),1],
    z=completeSetTSNE[:len(actualSets),2],
    mode='markers',
    marker=dict(
        size=7,
        color=marker_colors,                # set color to an array/list of desired values
        opacity=0.1,
        symbol='circle'
    )
)

# Plot
if len(medoidSets) > 0:
    data = [traceStandard, traceActual, traceMedoids]
else:
    data = [traceStandard, traceActual]

layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=data, layout=layout)
fig.show()

## 2D Visualization

In [ ]:
fig, ax = plt.subplots(figsize = (14, 10))

for i in range(len(std_dev)):
    coordinates = off[len(std_dev)-1-i] 
    x = [point[0] for point in coordinates]
    y = [point[1] for point in coordinates]  

    ax.scatter(x, y, s = 100,  edgecolor = "black", label="std_dev = {}".format(std_dev[len(std_dev)-1-i]))
    ax.scatter(0,0, s=100, color="black", marker='*')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.axhline(y=0, color='black', linestyle='--', label = "Global minima")
ax.axhline(y=10, color='grey', linestyle='--')
ax.axvline(x=0, color='black', linestyle='--')
ax.axvline(x=10, color='grey', linestyle='--',label = "Starting point")

ax.set_title("Offspring generated with different standard deviation values", fontsize=20)
ax.legend()

plt.show()